# The Float Class

The float class is an abbreviation for a floating point number and is Pythons default representation for a number that has a non-integer component. The floating point number is displayed in the decimal numbering system using ten characters 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 however under the hood is physically stored using bits which have the combination 0 and 1. 4 bytes are used to store the number using the IEEE-754 convention:

## Initialisation Signature

The docstring of the initialisation signature for the float class can be viewed by inputting:

In [17]:
? float

Init signature:  float(x=0, /)
Docstring:      Convert a string or number to a floating point number, if possible.
Type:           type
Subclasses:     

Notice that it has an input argument x which has a default value of 0. This means that the default floating point number is 0.0, notice the . indicating the decimal point:

In [18]:
float()

0.0

The input argument x is preceded by / meaning it has to be specified positionally. x can be an integer:

In [20]:
float(1)

1.0

In [21]:
float(True)

1.0

String of a floating point number:

In [27]:
float('3')

3.0

In [28]:
float('3.14')

3.14

In [26]:
float('2.97e8')

297000000.0

Or a floating point number:

In [29]:
float(3.14)

3.14

In [30]:
float(2.97e8)

297000000.0

The floating point number is a fundamental data type and normally instantiated using the number with the decimal point (fixed format):

In [31]:
3.14

3.14

For very small and very large numbers an exponent to the base 10 (scientific notation) is normally used, the radius of a hydrogen atom, radius of a human and radius of the sun are for example:

In [56]:
rhydrogen = 1.2e-10
rhydrogen

1.2e-10

In [57]:
rhuman = 1
rhuman

1

In [58]:
rsun = 6.957e8
rsun

695700000.0

## Format Specifiers

To understand the difference in the numbers above a formatted string can be used with a fixed format specifier:

In format specifier: 
* 0 is an instruction to show leading zeros.
* w is the character width of the number in the formatted string and includes the decimal point.
* p is the precision, i.e. the number of digits past the decimal point.
* f is used to represent the fixed format.

If for example the three radii explored above are shown with 20 digits before and after the decimal point, the difference in magnitude between the very small radius and the very large radius can be seen:

In [69]:
f'{rhydrogen:041.20f}'

'00000000000000000000.00000000012000000000'

In [70]:
f'{rhuman:041.15f}'

'0000000000000000000000001.000000000000000'

In [71]:
f'{rsun:041.15f}'

'0000000000000000695700000.000000000000000'

In the physical world, every number measured has an associated error. The sun for example is constituted of an exceedingly large number of hydrogen atoms:

In [89]:
numhydrogen = rsun / rhydrogen 
numhydrogen

5.7975e+18

In [90]:
f'{numhydrogen:041.15f}'

'0000005797500000000000000.000000000000000'

Because this is such a large number... its associated error is likely to be larger than the width of more than a million million hydrogen atoms. Naturally the error in the seperate measurement involving the width of a single hydrogen atom is much smaller.

Quoting the numbers to the above number of zeros doesn't physically make sense as they are not measured to such accuracy and it is not human readible to read that many zeros. Instead the format specifier can be changed to e to give:

In format specifier: 
* 0 is an instruction to show leading zeros (unspecified).
* w is the character width of the number in the formatted string and includes the decimal point (unspecified).
* p is the precision, i.e. the number of digits past the decimal point.
* e is used to represent the exponent (scientific notation).

In [91]:
f'{rsun:.3e}'

'6.957e+08'

In [92]:
f'{rhydrogen:.3e}'

'1.200e-10'

If the division is again examined:

In [99]:
f'{rsun / rhydrogen:.3e}'

'5.798e+18'

Notice that the mantissa of the first number is divided by the second number:

In [94]:
6.957 / 1.200

5.7975

And there is a subtraction of exponents, the exponent of the first number subtracts the exponent of the second number:

In [95]:
8 - (-10)

18

Lets examine the inverse operation multiplication:

In [96]:
f'{rhydrogen:.3e}'

'1.200e-10'

In [97]:
f'{numhydrogen:.3e}'

'5.798e+18'

If multiplication is examined:

In [100]:
f'{rhydrogen * numhydrogen:.3e}'

'6.957e+08'

The mantissa of the two numbers are multiplied:

In [102]:
1.200 * 5.798

6.9576

And the exponents are added:

In [103]:
-10 + 18

8

Operations involving addition or subtraction of the small number from the large number result in the large  number being unchanged as the small number is orders of magnitude smaller than the larger numbers error.

## Binary Encoding

Although a floating point number is displayed in decimal:

In [106]:
0.1

0.1

Under the hood a floating point number is encoded in binary using the IEEE754 technical standard for floating point numbers. The pickle module is used for serialisation of a number:

In [104]:
import pickle

The pickle modules dump string can be used to dump a Python object into a serialised byte string. The pickle module tends to add some additional information to the byte string and this can be removed by indexing into the relevant part of interest. For convenience this operation will be placed in a function:

In [111]:
def float2bytes(f1)
    return bin(int(pickle.dumps(f1)[12:20].hex(), base=16)).removeprefix('0b').zfill(64)

In IEEE754 there is:
* 1 bits for the sign
* 8 bits for the biased exponent
* 23 bits for the fraction

In [114]:
from collections import namedtuple

In [206]:
FloatAsByte = namedtuple('FloatAsByte', field_names=('sign', 'biasedexponent', 'fraction'))
FloatAsByte2 = namedtuple('FloatAsByte', field_names=('sign', 'exponent', 'fraction'))

def float2bytestuple(f1):
    string = bin(int(pickle.dumps(f1)[12:20].hex(), base=16)).removeprefix('0b').zfill(64)
    return FloatAsByte(string[0], string[1:9], string[9:])

def float2bytestuple2(f1):
    string = bin(int(pickle.dumps(f1)[12:20].hex(), base=16)).removeprefix('0b').zfill(64)
    if string[0] == '0':
        sign = '+'
    else:
        sign = '-'
    
    bias = 127    
    biasedexponent = int('0b' + string[1:9], base=2) 
    print(biasedexponent)
    rawexponent = biasedexponent - bias
    print(rawexponent)
    stringexponent = bin(rawexponent).removeprefix('0b').zfill(8)   
    fraction = '1.' + string[9:]
    return FloatAsByte2(sign, stringexponent, fraction)

And the binary

In [207]:
float2bytes(0.1)

'0011111110111001100110011001100110011001100110011001100110011010'

In [208]:
float2bytestuple(0.1)

FloatAsByte(sign='0', biasedexponent='00000001', fraction='0011001100110011001100110011001100110011001101000101110')

In [205]:
float2bytestuple2(0.1)

0
-127


FloatAsByte(sign='+', exponent='-0b1111111', fraction='1.0000000010001110011111110111001100110011001100110011001')

In [185]:
0b01111111

127

In [223]:
pickle.dumps(0.1).hex()

'8004950a00000000000000473fb999999999999a2e'

In [218]:
bin(int(pickle.dumps(0.1)[13:22].hex(), base=16))

'0b1011100110011001100110011001100110011001100110011001101000101110'

In [225]:
pickle.format_version

'4.0'